In [1]:
import pandas as pd

# Load the CSV
file_path = '/Users/azadehbayani/Documents/Azadé/LabTNS/Projects/CharlsonCodes/ModifiedVersion/Data/ElixhauserMapping.csv'
df = pd.read_csv(file_path, delimiter=',', skip_blank_lines=False)


In [2]:
df

,ElixhauserComorbidity,ICD-10-CM Code,ICD-10-CA Code,ICD-10-CA Label,Unnamed: 4
0,Chf (congestive heart failure),I099,I099,Rheumatic heart disease,NaN
1,Chf (congestive heart failure),I110,I110,Hypertensive heart disease with (congestive) h...,NaN
2,Chf (congestive heart failure),I130,I130,Hypertensive heart and renal disease with (con...,NaN
3,Chf (congestive heart failure),I132,I132,Hypertensive heart and renal disease with both...,NaN
4,Chf (congestive heart failure),I255,I255,Ischaemic cardiomyopathy,NaN
...,...,...,...,...,...
506,Depre (depression),F33,F334,Recurrent depressive disorder,currently in remission
507,Depre (depression),F341,F341,Dysthymia,NaN
508,Depre (depression),F412,F412,Mixed anxiety and depressive disorder,NaN
509,Depre (depression),F432,F432,Adjustment disorders,NaN


In [7]:

# Expand rows with multiline cells
df_expanded = df.assign(**{
    col: df[col].str.split('\n') if df[col].dtype == 'object' else df[col]
    for col in df.columns
}).explode(df.columns[0])  # Explode the first column by default

# Split rows with mixed comma-separated values in a single cell
df_cleaned = df_expanded.assign(
    split_data=df_expanded.iloc[:, 0].str.split(',')
).explode('split_data')

processed_rows = []
for index, row in df_cleaned.iterrows():
    # Ensure that 'split_data' is a string before attempting to split
    if isinstance(row['split_data'], str):
        values = row['split_data'].split(',')
        if len(values) >= 4:
            processed_rows.append({
                'Elixhauser Comorbidity': values[0].strip(),
                'ICD-10-CM Code': values[1].strip(),
                'ICD-10-CA Code': values[2].strip(),
                'ICD-10-CA Label': ','.join(values[3:]).strip()  # Handle labels with commas
            })
    else:
        # Handle the case where 'split_data' is not a string (e.g., NaN or other float values)
        processed_rows.append({
            'Elixhauser Comorbidity': None,
            'ICD-10-CM Code': None,
            'ICD-10-CA Code': None,
            'ICD-10-CA Label': None
        })

# Create the new DataFrame with processed data
processed_df = pd.DataFrame(processed_rows)



In [13]:
data = df['ElixhauserComorbidity'].iloc[172]
data
# Step 1: Split the data into rows based on line breaks
rows = data.strip().split("\n")
rows

['Hypc (hypertension,I11,I11,Hypertensive heart disease\r',
 'Hypc (hypertension']

In [14]:

# Step 2: Split each row into fields based on commas
data_list = [row.split(",") for row in rows]

# Step 3: Handle varying lengths in rows
# If some rows have extra commas, truncate or fill with None
max_columns = max(len(row) for row in data_list)
data_list = [row + [None] * (max_columns - len(row)) for row in data_list]

data_list

[['Hypc (hypertension', 'I11', 'I11', 'Hypertensive heart disease\r'],
 ['Hypc (hypertension', None, None, None]]